![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI&file=Grounding+Overview+-+Vertex+AI+Search.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Grounding%20Overview%20-%20Vertex%20AI%20Search.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FGrounding%2520Overview%2520-%2520Vertex%2520AI%2520Search.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Grounding%20Overview%20-%20Vertex%20AI%20Search.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Grounding%20Overview%20-%20Vertex%20AI%20Search.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Grounding Overview - Vertex AI Search


todo:
- test search app
- include console links
- note on managing metadata files
- include cleanup section for app and datastore


**SPECIAL NOTES**

This workflow using Vertex AI Agent Builder To Build a search experience with Vertesx AI Search.  If this tool has not bee previously used the part of the setup may need to be completed in the console prior to running this notebook. See [Before you begin](https://cloud.google.com/generative-ai-app-builder/docs/before-you-begin).


---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.storage', 'google-cloud-storage'),
    ('google.cloud.discoveryengine', 'google-cloud-discoveryengine')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [5]:
!gcloud services enable discoveryengine.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [6]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

Inputs

In [7]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [10]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'grounding-overview'

# make this the gcs bucket for storing files
GCS_BUCKET = PROJECT_ID 

# vertex search location
VS_LOCATION = 'global'

Packages

In [27]:
import requests
import base64
import json
import hashlib

from google.cloud import storage

import google.cloud.discoveryengine_v1 as discoveryengine
import google.cloud.discoveryengine_v1alpha as discoveryengine_alpha

Clients

In [61]:
# vertex ai agent builder
API_ENDPOINT = dict(api_endpoint = (f'{VS_LOCATION}-' if VS_LOCATION != 'global' else '') + 'discoveryengine.googleapis.com')
datastore_client = discoveryengine.DataStoreServiceClient(client_options = API_ENDPOINT)
document_client = discoveryengine.DocumentServiceClient(client_options = API_ENDPOINT)
engine_client = discoveryengine.EngineServiceClient(client_options = API_ENDPOINT)
search_client = discoveryengine.SearchServiceClient(client_options = API_ENDPOINT)

# gcs client: assumes bucket already exists
gcs = storage.Client(project = PROJECT_ID)
bucket = gcs.bucket(GCS_BUCKET)

---
## Prompt And Context

The [official rules of baseball](https://img.mlbstatic.com/mlb-images/image/upload/mlb/wqn5ah4c3qtivwx3jatm.pdf), a pdf that is updated annually with the latest changes to the game and published by MLB.


In [12]:
prompt = "what are the dimensions of first base in baseball?"

In [16]:
url = 'https://img.mlbstatic.com/mlb-images/image/upload/mlb/wqn5ah4c3qtivwx3jatm.pdf'
# get the pdf
context_bytes = requests.get(url).content
context_base64 = base64.b64encode(context_bytes).decode('utf-8')

---
## Store Document(s) In GCS

In [106]:
# store pdf in gcs
file_blob = bucket.blob(f"{SERIES}/{EXPERIMENT}/{url.split('/')[-1]}")
file_blob.upload_from_string(context_bytes, content_type = 'application/pdf')

---
## Prepare Document(s) For The Vertex Agent Builder Data Store

There are multiple ways to [prepare data for ingesting](https://cloud.google.com/generative-ai-app-builder/docs/prepare-data) depending on location, volume, how often it will change, and type (website, unstructured, strutred, media, third-party (Slack, ServiceNow, ...), Healthcare FHIR, ....).

Here, the data will be prepared as **Unstructured data in GCS storage**.  Files an be imported as:
- **single file** at GCS URI
- **multiple files** in a GCS 'folder'.  Note that import is nnot recursive so subfolder will not be imported.  For this case or even multiple buckets see the next option:
- **Any number of files** at a single or multiple folders and buckets can be imported with a **metadata** file.  A JSON lines file with one line per file that include the document id and uri as well as optional metadata.


In [107]:
for blob in bucket.list_blobs(prefix = f"{SERIES}/{EXPERIMENT}/{url.split('/')[-1]}"):
    print(blob.name)

applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf


### Document ID's

The name of a file could make a great id.  But what happens if multiple file names exist but in different folders?  To make this managable across files, folders and buckets, adopting a hash of the full file path could be a great practice.  This function is designed to convert the full file path into a hash of length 63.

In [108]:
def generate_id(name):
    hasher =hashlib.sha256()
    hasher.update(name.encode('utf-8'))
    return hasher.hexdigest()[0:63]

In [110]:
generate_id(bucket.name+'/'+blob.name)

'b48b4704ced27a1e9bdee5b1d90479abdb488b2df9fa0b93d554ea250039e18'

### Create Metadata

At a minimum the document `id` and the `content` needs to be provided but optional metadata can also provided as shown here with `structData`:

In [111]:
metadata = []
file_types = ['pdf', 'docx', 'txt', 'html', 'pptx']
content_type = dict(
  pdf = 'application/pdf',
  txt = 'text/plain',
  html = 'text/html',
  docx = 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
  pptx = 'application/vnd.openxmlformats-officedocument.presentationml.presentation'
)
for blob in bucket.list_blobs(prefix = f"{SERIES}/{EXPERIMENT}/{url.split('/')[-1]}"):
    folder = blob.name.split('/')[0]
    filename = blob.name.split('/')[-1]
    filetype = blob.name.split('.')[-1].lower()
    filepath = '/'.join(blob.name.split('/')[0:-1])
    if filetype in file_types:
        json_data = dict(
            id = generate_id(blob.name),
            structData = dict(
                title= filename,
                path = filepath,
                location = bucket.name
            ),
            content = dict(
                mimeType = content_type[filetype],
                uri = f'gs://{blob.bucket.name}/{blob.name}'
            )
        )
        metadata.append(json.dumps(json_data))   


In [112]:
metadata

['{"id": "73bee30d37112e8e829720fe2f4a3ff0b450c2a51cd9fe03b5beeea277eed67", "structData": {"title": "wqn5ah4c3qtivwx3jatm.pdf", "path": "applied-genai/grounding-overview", "location": "statmike-mlops-349915"}, "content": {"mimeType": "application/pdf", "uri": "gs://statmike-mlops-349915/applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf"}}']

### Write Metadata File To GCS

In [113]:
metadata_blob = bucket.blob(f'{SERIES}/{EXPERIMENT}/{SERIES}-{EXPERIMENT}.json')
with metadata_blob.open('w') as f:
    for m in metadata:
        f.write(m + '\n')

In [114]:
f"gs://{bucket.name}/{blob.name}"

'gs://statmike-mlops-349915/applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf'

---
## Create A Search Data Store

[Creating a search data store](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es) depends on the type of data, in this case an [import from cloud storage](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es#cloud-storage) using a metadata file in JSON lines format. 

- [Discoveryengine Python Data Store Client](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.data_store_service.DataStoreServiceClient)

In [115]:
VS_DATASTORE_ID = f"{SERIES}-{EXPERIMENT}"

### Check For Existing Data Store (and retrieve)

In [116]:
try:
    datastore = datastore_client.get_data_store(
        name = datastore_client.collection_path(
            project = PROJECT_ID,
            location = VS_LOCATION,
            collection = 'default_collection'
        ) + f'/dataStores/{VS_DATASTORE_ID}'
    )
    ds_exist = True
except Exception as err:
    ds_exist = False
    
ds_exist

True

### Create New Data Store (if needed)

In [117]:
if not ds_exist:
    ds_create = datastore_client.create_data_store(
        parent = datastore_client.collection_path(
            project = PROJECT_ID,
            location = VS_LOCATION,
            collection = 'default_collection'
        ),
        data_store = discoveryengine.DataStore(
            display_name = f"{SERIES}-{EXPERIMENT}",
            industry_vertical = discoveryengine.IndustryVertical.GENERIC,
            solution_types = [discoveryengine.SolutionType.SOLUTION_TYPE_SEARCH],
            content_config = discoveryengine.DataStore.ContentConfig.CONTENT_REQUIRED,
            #document_processing_config = discoveryengine.DocumentProcessingConfig(
            #    #chunking_config = ,
            #    default_parsing_config = discoveryengine.DocumentProcessingConfig.ParsingConfig.DigitalParsingConfig,
            #    parsing_config_overrides = [
            #        {'pdf' :discoveryengine.DocumentProcessingConfig.ParsingConfig.LayoutParsingConfig}
            #    ]
            #)
        ),
        data_store_id = VS_DATASTORE_ID
    )
    response = ds_create.result()
    print(ds_create.operation.name)

### Get The Data Store ID

In [118]:
datastore = datastore_client.get_data_store(
    name = datastore_client.collection_path(
        project = PROJECT_ID,
        location = VS_LOCATION,
        collection = 'default_collection'
    ) + f'/dataStores/{VS_DATASTORE_ID}'
)
datastore

name: "projects/1026793852137/locations/global/collections/default_collection/dataStores/applied-genai-grounding-overview"
display_name: "applied-genai-grounding-overview"
industry_vertical: GENERIC
solution_types: SOLUTION_TYPE_SEARCH
default_schema_id: "default_schema"
content_config: CONTENT_REQUIRED
create_time {
  seconds: 1724189228
  nanos: 173562000
}

In [119]:
datastore.name

'projects/1026793852137/locations/global/collections/default_collection/dataStores/applied-genai-grounding-overview'

In [121]:
datastore_id = datastore_client.parse_data_store_path(datastore.name)
datastore_id

{'project': '1026793852137',
 'location': 'global/collections/default_collection',
 'data_store': 'applied-genai-grounding-overview'}

### List Documents If Prior Data Store

In [125]:
if ds_exist:
    doc_exist = False
    for doc in document_client.list_documents(
        parent = document_client.branch_path(
            project = PROJECT_ID,
            location = VS_LOCATION,
            data_store = datastore_id['data_store'],
            branch = 'default_branch'
        )
    ):
        print(doc.content.uri)
        if doc.content.uri == f"gs://{bucket.name}/{file_blob.name}":
            doc_exist = True
            break

doc_exist

gs://statmike-mlops-349915/applied-genai/grounding-overview/wqn5ah4c3qtivwx3jatm.pdf


True

### Import Documents (if missing)

- [Discoveryengine Python Document Service Client](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.document_service.DocumentServiceClient)

In [127]:
if not doc_exist:
    doc_import = document_client.import_documents(
        request = discoveryengine.ImportDocumentsRequest(
            parent = document_client.branch_path(
                project = PROJECT_ID,
                location = VS_LOCATION,
                data_store = datastore_id['data_store'],
                branch = 'default_branch'
            ),
            gcs_source = discoveryengine.GcsSource(
                input_uris = [f"gs://{bucket.name}/{metadata_blob.name}"],
                data_schema = 'document'
            ),
            reconciliation_mode = discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL
        )
    )
    response = doc_import.result()
    operation_metadata = discoveryengine.ImportDocumentsMetadata(doc_import.metadata)
    print(operation_metadata)

---
## Create A Search App

The [Relationship between apps and data stores](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest#app-store-relationship) allows one or more apps to search and retrieve from or more (blended search) data stores.

Here we [creatre an app](https://cloud.google.com/generative-ai-app-builder/docs/create-engine-es) that uses the data store created above.

- [Discoveryengine Python Engine Service Client](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.engine_service.EngineServiceClient)

In [82]:
VS_APP_ID = f"{SERIES}-{EXPERIMENT}"

### Check For Existing Search App

In [130]:
try:
    app = engine_client.get_engine(
        name = engine_client.engine_path(
            project = PROJECT_ID,
            location = VS_LOCATION,
            collection = 'default_collection',
            engine = VS_APP_ID
        )
    )
    app_exist = True
except Exception as err:
    app_exist = False
    
app_exist

True

### Create New Search App (if needed)

In [132]:
if not app_exist:
    app_create = engine_client.create_engine(
        parent = engine_client.collection_path(
            project = PROJECT_ID,
            location = VS_LOCATION,
            collection = 'default_collection'
        ),
        engine = discoveryengine.Engine(
            display_name = VS_APP_ID,
            industry_vertical = discoveryengine.IndustryVertical.GENERIC,
            solution_type = discoveryengine.SolutionType.SOLUTION_TYPE_SEARCH,
            search_engine_config = discoveryengine.Engine.SearchEngineConfig(
                search_tier = discoveryengine.SearchTier.SEARCH_TIER_ENTERPRISE,
                search_add_ons = [discoveryengine.SearchAddOn.SEARCH_ADD_ON_LLM],
            ),
            data_store_ids = [datastore_id['data_store']],
        ),
        engine_id = VS_APP_ID
    )
    response = app_create.result()
    operation_metadata = discoveryengine.CreateEngineMetadata(app_create.metadata)
    print(operation_metadata)
    print(app_create.operation.name)

---
## Get Search Results 

[Get search results](https://cloud.google.com/generative-ai-app-builder/docs/preview-search-results) using the API.

---
## Get Answers With Follow-Ups

[Get answers and follow-ups](https://cloud.google.com/generative-ai-app-builder/docs/answer)

---
## Get Snippets And Extractive Segments

[Get Snippets And Extractive Segments](https://cloud.google.com/generative-ai-app-builder/docs/snippets)

---
## Get Search Summaries

[Get Search Summaries](https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries)